# Amazon Bedrock Guardrails と AgentCore Memory を使用した会話の保護

## 概要

このチュートリアルでは、Amazon Bedrock Guardrails と AgentCore Memory を統合して、安全な会話エージェントを作成する方法を説明します。インタラクション間で会話コンテキストを維持しながら、機密コンテンツをフィルタリングするエージェントを構築します。

### チュートリアル詳細

| 項目              | 詳細                                                             |
|:------------------|:-----------------------------------------------------------------|
| チュートリアルタイプ | Guardrails / Memory 統合                                        |
| エージェントタイプ   | 保護機能付き Memory 対応エージェント                              |
| エージェントフレームワーク | Strands Agents                                              |
| LLM モデル         | Anthropic Claude Haiku 4.5                                      |
| 主な機能           | Guardrails、Memory 統合、コンテンツフィルタリング                  |
| 難易度             | 中級                                                             |
| 使用 SDK          | Amazon Bedrock Python SDK および Bedrock Memory SDK              |

### 学習内容

このチュートリアルでは、以下を学習します：
1. エージェント用の Memory リソースを作成する方法
2. コンテンツフィルタリング付きの Amazon Bedrock Guardrails を実装する方法
3. Guardrails と Memory 機能を組み合わせたカスタムフックを構築する方法
4. 安全な会話履歴を選択的に保存する方法
5. 様々なタイプのコンテンツでセキュアエージェントをテストする方法

### アーキテクチャ

この例は、安全な会話のための Guardrails と Memory の統合を示しています：

<div style="text-align:left">
    <img src="guardrails_memory_flow.png" width="90%"/>
</div>

## 0. 前提条件

このチュートリアルを実行するには、以下が必要です：
* Python 3.10 以上
* AgentCore Memory および Amazon Bedrock へのアクセス権限を持つ AWS 認証情報
* Amazon Bedrock モデルアクセス（Claude 3.7 Sonnet）
* Amazon Bedrock Memory SDK

まず、必要なライブラリをインストールしましょう：

In [ ]:
!pip install -qr requirements.txt

In [ ]:
# Imports
import os
import boto3
import uuid
import logging
from typing import Dict
from strands import Agent
from strands.models import BedrockModel
from bedrock_agentcore.memory import MemoryClient
from botocore.exceptions import ClientError
from strands.hooks import AgentInitializedEvent, HookProvider, HookRegistry, MessageAddedEvent
from strands.experimental.hooks import AfterModelInvocationEvent

# Configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(name)s: %(message)s'
)
logger = logging.getLogger("secure-agent")
REGION = os.getenv('AWS_REGION', 'us-west-2') # AWS region for the agent
bedrock_client = boto3.client('bedrock', region_name=REGION)
bedrock_runtime_client = boto3.client('bedrock-runtime', region_name=REGION)
memory_client = MemoryClient(region_name=REGION)

## 1. Amazon Bedrock Guardrails の作成

このセクションでは、エージェントにコンテンツ安全ポリシーを適用するための Guardrail を作成します。Guardrails は、ユーザー入力とモデル出力の両方に適用できる安全フィルターとして機能します。この例では、2つの特定のポリシーを持つ Guardrail を作成します：

1. **入力フィルタリング**：ユーザーからの侮辱的な言葉をブロック
2. **出力フィルタリング**：モデルが政治的なトピックについて議論することを防止

このアプローチは、会話の両方向から異なるタイプの問題のあるコンテンツに対して Guardrails がどのように保護できるかを示しています。入力フィルターは礼儀正しい会話環境を維持するのに役立ち、出力フィルターはモデルが潜在的にセンシティブなトピックについて議論しないことを保証します。

最終的な目標は、不適切なメッセージが Memory に保存されることを防ぎ、適切なコンテンツのみが将来のコンテキストとして保存されるようにすることです。

In [ ]:
# Unique identifier for this request
unique_id = str(uuid.uuid4())[:6]

# Define guardrail configuration
guardrail_name = f"SecureConversationGuardrail_{unique_id}"
guardrail_description = "Blocks insults in input and political content in output"

try:
    # Create the guardrail
    response = bedrock_client.create_guardrail(
        name=guardrail_name,
        description=guardrail_description,
        # Block insults in input
        contentPolicyConfig={
            'filtersConfig': [
                {
                    'type': 'INSULTS',
                    'inputStrength': 'MEDIUM',
                    'outputStrength': 'MEDIUM',
                    'inputModalities': ['TEXT'],
                    'outputModalities': ['TEXT'],
                    'inputAction': 'BLOCK',
                    'outputAction': 'NONE',
                    'inputEnabled': True,
                    'outputEnabled': False
                }
            ],
            'tierConfig': {
                'tierName': 'CLASSIC'
            }
        },
        # Block political content in output
        topicPolicyConfig={
            'topicsConfig': [
                {
                    'name': 'Politics',
                    'definition': 'Content related to political leaders, elections, political parties, or government affairs',
                    'examples': [
                        'Who is the current president?',
                        'Tell me about the upcoming election',
                        'Explain the political situation in Congress'
                    ],
                    'type': 'DENY',
                    'inputAction': 'NONE',
                    'outputAction': 'BLOCK',
                    'inputEnabled': False,
                    'outputEnabled': True
                }
            ],
            'tierConfig': {
                'tierName': 'CLASSIC'
            }
        },
        blockedInputMessaging="I'm sorry, but your message contains inappropriate language. Please rephrase your question without insults.",
        blockedOutputsMessaging="I apologize, but I cannot provide information on political topics. Is there something else I can help you with?",
    )
    
    # Store guardrail ID for later use
    guardrail_id = response['guardrailId']
    guardrail_arn = response['guardrailArn']
    guardrail_version = "DRAFT"  # New guardrails are created as DRAFT
    
    print(f"✅ Created guardrail: {guardrail_id} (ARN: {guardrail_arn})")
    
except Exception as e:
    print(f"❌ Error creating guardrail: {e}")
    # If the guardrail already exists, try to find its ID
    try:
        response = bedrock_client.list_guardrails()
        existing_guardrail = next((g for g in response['guardrailSummaries'] 
                                  if g['name'] == guardrail_name), None)
        if existing_guardrail:
            guardrail_id = existing_guardrail['guardrailId']
            guardrail_version = "DRAFT"  # Use DRAFT version
            print(f"Using existing guardrail: {guardrail_id}")
    except Exception as list_error:
        print(f"❌ Error listing guardrails: {list_error}")
        guardrail_id = None
        guardrail_version = None


## 2. Memory リソースの作成

このセクションでは、エージェントが会話履歴を保存するための Memory リソースを作成します。Memory により、エージェントは過去のインタラクションを思い出し、コンテキストを維持し、より一貫性のある応答を提供できます。Memory と Guardrails を組み合わせることで、適切なコンテンツのみが将来の参照用に保存されることを保証できます。

この例では、追加のストラテジーを持たないシンプルな短期 Memory リソースを作成します。これは、セッション内で会話コンテキストを維持するのに最適です。Memory は Guardrail チェックを通過したメッセージを保存し、不適切なコンテンツがフィルタリングされることを保証します。

In [ ]:
memory_name = f"SecureAgentMemory_{unique_id}"

try:
    # Create memory resource without strategies (thus only access to short-term memory)
    memory = memory_client.create_memory_and_wait(
        name=memory_name,
        strategies=[],  # No strategies for short-term memory
        description="Short-term memory for personal agent with guardrails",
        event_expiry_days=7,
    )
    memory_id = memory['id']
    logger.info(f"✅ Created memory: {memory_id}")
except ClientError as e:
    logger.info(f"❌ ERROR: {e}")
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID
        memories = memory_client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Show any errors during memory creation
    logger.error(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    # Cleanup on error - delete the memory if it was partially created
    if 'memory_id' in locals() and memory_id:
        try:
            memory_client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.error(f"Failed to clean up memory: {cleanup_error}")


## 3. Bedrock Guardrails、Strands、AgentCore Memory の統合

このセクションでは、Guardrails と Memory 機能を統合するカスタムフックを作成します。実装では以下を行います：

1. Amazon Bedrock Guardrails を使用してユーザー入力とモデル出力の両方をチェック
2. 不適切なコンテンツを安全な代替コンテンツに置き換え
3. Guardrail チェックを通過したメッセージのみを Memory に保存
4. エージェント初期化時に過去の会話コンテキストを Memory から取得

このアプローチにより、エージェントは Memory 機能の恩恵を受けながら、クリーンな会話履歴を維持できます。必要なコンポーネントを構築しましょう：

In [ ]:
class GuardrailsEvaluator:
    """Reusable guardrails evaluation utility."""
    
    def __init__(self, guardrail_id: str, guardrail_version: str):
        """Initialize the guardrails evaluator.
        
        Args:
            guardrail_id: The ID of the guardrail to use
            guardrail_version: The version of the guardrail (e.g., "DRAFT")
        """
        self.guardrail_id = guardrail_id
        self.guardrail_version = guardrail_version
    
    def evaluate_content(self, content: str, source: str) -> Dict:
        """Evaluate content using Bedrock Guardrails and return result.
        
        Args:
            content: The text content to evaluate
            source: The source type ("INPUT" or "OUTPUT")
            
        Returns:
            Dict containing guardrail evaluation results
        """
        try:
            logger.warning(f"⏳ CHECKING {source}: '{content[:30]}...'")
            
            response = bedrock_runtime_client.apply_guardrail(
                guardrailIdentifier=self.guardrail_id,
                guardrailVersion=self.guardrail_version,
                source=source,
                content=[{"text": {"text": content}}]
            )
            
            action = response.get('action')
            logger.warning(f"🔍 GUARDRAIL ACTION: {action}")
            
            return response
        except Exception as e:
            logger.error(f"❌ Guardrail evaluation failed: {e}")
            return {"error": str(e)}


class GuardrailsMemoryHookProvider(HookProvider):
    """Hook provider that combines guardrails enforcement with memory storage."""
    
    def __init__(
        self, 
        memory_client: MemoryClient, 
        memory_id: str, 
        actor_id: str, 
        session_id: str,
        guardrails_evaluator: GuardrailsEvaluator
    ):
        """Initialize the hook provider.
        
        Args:
            memory_client: Client for memory storage
            memory_id: ID of the memory to use
            actor_id: ID of the actor
            session_id: ID of the current session
            guardrails_evaluator: Evaluator for content validation
        """
        # Memory properties
        self.memory_id = memory_id
        self.actor_id = actor_id
        self.session_id = session_id
        
        # Guardrails evaluator
        self.evaluator = guardrails_evaluator
        
        # Message tracking system
        self.blocked_outputs = set()  # Model outputs that failed guardrail checks

    def after_model_invocation(self, event: AfterModelInvocationEvent) -> None:
        """Check model output with guardrails and replace if needed.
        
        Args:
            event: Event containing the model response
        """
        # Skip if model invocation failed
        if event.exception is not None or event.stop_response is None:
            logger.error("⚠️ Model invocation failed, skipping guardrail check")
            return
        
        logger.info("🔍 AfterModelInvocationEvent: Checking model output")
        
        # Extract message from the model response
        message = event.stop_response.message
        
        # Extract content
        if isinstance(message.get("content"), list):
            content = "".join(block.get("text", "") for block in message.get("content", []))
        else:
            content = str(message.get("content", ""))
        
        content_id = hash(content)
        
        # Check against guardrails
        result = self.evaluator.evaluate_content(content, "OUTPUT")
        
        # Handle guardrail violations
        if result.get("action") == "GUARDRAIL_INTERVENED":
            logger.warning("⛔ ASSISTANT MESSAGE BLOCKED BY GUARDRAILS")
            
            # Mark this output as blocked
            self.blocked_outputs.add(content_id)
            
            # Get the guardrail-provided alternative if available
            replacement_content = None
            if "outputs" in result and result["outputs"] and len(result["outputs"]) > 0:
                if "text" in result["outputs"][0]:
                    replacement_content = result["outputs"][0]["text"]
            
            # Fall back to generic message if no replacement provided
            if not replacement_content:
                replacement_content = "I apologize, but I cannot provide the requested information as it would violate our content policies."
            
            # Update the message content - THIS WILL CHANGE WHAT THE USER SEES
            if isinstance(message.get("content"), list):
                message["content"] = [{"text": replacement_content}]
            else:
                message["content"] = replacement_content
            
            logger.info(f"⚠️ Replaced assistant message with guardrail response: {replacement_content[:30]}...")
    
    def on_message_added(self, event: MessageAddedEvent) -> None:
        """Store messages in memory if they pass checks.
        
        Args:
            event: Event containing the message to store
        """
        # Get the message
        message = event.message
        
        # Skip if no message
        if not message:
            return
        
        # Extract content
        if isinstance(message.get("content"), list):
            content = "".join(block.get("text", "") for block in message.get("content", []))
        else:
            content = str(message.get("content", ""))
        
        role = message.get("role", "")
        content_id = hash(content)
        
        # Skip system messages
        if role == "system":
            return
        
        # Skip if this specific message was blocked (only applies to assistant messages)
        if role == "assistant" and content_id in self.blocked_outputs:
            logger.info(f"⛔ Skipping memory storage for blocked {role} message")
            return
        
        # Store the message in memory
        try:
            logger.info(f"💾 Storing {role} message in memory (passed guardrail check)")
            memory_client.create_event(
                memory_id=self.memory_id,
                actor_id=self.actor_id,
                session_id=self.session_id,
                messages=[(content, role)]
            )
        except Exception as e:
            logger.error(f"❌ Memory save error: {e}")
    
    def on_agent_initialized(self, event: AgentInitializedEvent) -> None:
        """Load recent conversation history when agent starts.
        
        Args:
            event: Event containing the agent that was initialized
        """
        try:
            recent_turns = memory_client.get_last_k_turns(
                memory_id=self.memory_id,
                actor_id=self.actor_id,
                session_id=self.session_id,
                k=5
            )
            
            if recent_turns:
                context_messages = []
                for turn in recent_turns:
                    for message in turn:
                        role = message['role']
                        content = message['content']['text']
                        context_messages.append(f"{role}: {content}")
                
                context = "\n".join(context_messages)
                event.agent.system_prompt += f"\n\nRecent conversation:\n{context}"
                logger.info(f"✅ Loaded {len(recent_turns)} conversation turns")
                
        except Exception as e:
            logger.error(f"Memory load error: {e}")

    def register_hooks(self, registry: HookRegistry):
        """Register all hooks with the registry.
        
        Args:
            registry: The hook registry to register with
        """
        registry.add_callback(AfterModelInvocationEvent, self.after_model_invocation)
        registry.add_callback(MessageAddedEvent, self.on_message_added)
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)

## 4. エージェントの作成と設定

このセクションでは、これまでに構築したすべてのコンポーネント（Bedrock モデル、Guardrails 評価器、Memory 対応フックプロバイダー）を組み合わせて、セキュアな会話エージェントを作成します。この統合により、コンテンツポリシーを適用し、適切なコンテキストを保存しながら会話を維持できる完全なエージェントが作成されます。

In [ ]:
ACTOR_ID = "user_1"
SESSION_ID = "session_001"
bedrock_model = BedrockModel(model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0")

evaluator = GuardrailsEvaluator(
    guardrail_id=guardrail_id,
    guardrail_version=guardrail_version
)

def create_personal_agent():
    """Create personal agent with memory and guardrails"""
    agent = Agent(
        name="PersonalAssistant",
        model="global.anthropic.claude-haiku-4-5-20251001-v1:0",
        system_prompt="You are a helpful personal assistant. Be friendly and professional.",
        hooks=[GuardrailsMemoryHookProvider(memory_client, memory_id, ACTOR_ID, SESSION_ID, evaluator)],
        callback_handler=None
    )
    return agent

# Create agent
agent = create_personal_agent()
logger.info("✅ Personal agent created with memory and guardrails")

この実装により、以下の機能を持つセキュアエージェントが作成されます：

1. 初期化時に既存の会話コンテキストを Memory からロード
2. 処理前にユーザー入力を Guardrails でチェック
3. ユーザーに表示する前にモデル出力を Guardrails でチェック
4. 承認されたメッセージのみを将来のコンテキスト用に Memory に保存
5. 複数のインタラクションにわたって会話履歴を維持

Guardrails と Memory の組み合わせにより、エージェントはセキュアでありながらコンテキストを考慮した会話体験を提供できます。

## 5. セキュアエージェントのテスト

様々なタイプの入力でエージェントをテストして、Guardrails と Memory 統合が実際にどのように機能するかを確認しましょう。許容される入力と Guardrail の介入をトリガーする可能性のある入力の両方を試して、実装が正しく動作していることを確認します。

まず、Guardrail チェックとエージェント呼び出しを処理するヘルパー関数を作成しましょう：

In [ ]:
def process_with_guardrails(user_input):
    """Process user input with guardrails before sending to agent.
    
    Args:
        user_input: The text input from the user
        
    Returns:
        The agent response or guardrail rejection
    """
    # Check input against guardrails
    result = evaluator.evaluate_content(user_input, "INPUT")
    
    if result.get("action") == "GUARDRAIL_INTERVENED":
        # Get rejection message from guardrail
        if "outputs" in result and result["outputs"] and "text" in result["outputs"][0]:
            rejection_content = result["outputs"][0]["text"]
        else:
            rejection_content = "I cannot process that request."
        
        # Return rejection without calling agent
        print(rejection_content)
        return rejection_content
    else:
        # Input passed guardrails, proceed with agent call
        response = agent(user_input)
        print(response)
        return response

### テスト 1: 通常の会話

すべての Guardrails を通過するはずの通常の挨拶から始めましょう：

In [ ]:
print("Test 1: Normal greeting")
user_input = "I am dani."
process_with_guardrails(user_input)

### テスト 2: 侮辱的なコンテンツ（入力 Guardrail がトリガーされるべき）

入力 Guardrail によってブロックされるべき侮辱的な言葉を含む入力を試しましょう：

In [ ]:
print("\nTest 2: Insulting content (should trigger input guardrail)")
user_input = "You're a stupid assistant."
process_with_guardrails(user_input)

### テスト 3: 政治的なコンテンツ（出力 Guardrail がトリガーされるべき）

次に、政治に関する質問を試しましょう。これは入力 Guardrail を通過しますが、出力 Guardrail をトリガーするはずです：

In [ ]:
print("\nTest 3: Political question (should trigger output guardrail)")
user_input = "Who is the president of the US?"
process_with_guardrails(user_input)

### Memory 内容の確認

テスト後に Memory に何が保存されたか確認しましょう：

In [ ]:
# Check what's stored in memory
print("\n=== Memory Contents ===")
recent_turns = memory_client.get_last_k_turns(
    memory_id=memory_id,
    actor_id=ACTOR_ID,
    session_id=SESSION_ID,
    k=5  # Adjust k to see more or fewer turns
)

for i, turn in enumerate(recent_turns):
    print(f"\nTurn {i+1}:")
    for msg in turn:
        role = msg['role']
        content = msg['content']['text']
        print(f"- {role}: {content[:100]}...")

### テスト 4: Memory をテストするためのフォローアップ質問
エージェントが以前のコンテキストを覚えているかどうかを確認するために、フォローアップの質問をしましょう：

In [ ]:
agent = create_personal_agent()
print("\nTest 4: Follow-up to test memory")
user_input = "What's my name?"
process_with_guardrails(user_input)

## 6. クリーンアップ（オプション）

セキュアエージェントの実験が完了したら、このチュートリアルで作成したリソースをクリーンアップすることができます。このセクションでは、Guardrail と Memory リソースを削除する方法を示します。

In [ ]:
# Delete the memory resource
try:
    memory_client.delete_memory_and_wait(memory_id=memory_id)
    print(f"✅ Deleted memory resource: {memory_id}")
except Exception as e:
    print(f"❌ Error deleting memory: {e}")

# Delete the guardrail
try:
    bedrock_client.delete_guardrail(
        guardrailIdentifier=guardrail_id
    )
    print(f"✅ Deleted guardrail: {guardrail_id}")
except Exception as e:
    print(f"❌ Error deleting guardrail: {e}")

## まとめ

このチュートリアルでは、Amazon Bedrock Guardrails と AgentCore Memory 機能を組み合わせたセキュアな会話エージェントを構築しました。この実装では以下を実現しています：

1. Guardrails を使用して不適切なユーザー入力をフィルタリング
2. エージェントがセンシティブなトピックについて議論することを防止
3. 承認されたメッセージのみを Memory に保存
4. Memory を使用して会話コンテキストを維持し、ユーザー体験を向上

Guardrails と Memory を統合することで、コンテンツポリシーへのコンプライアンスを維持しながら、パーソナライズされたコンテキストを考慮した応答を提供する堅牢なエージェントを構築できます。このパターンは、追加の Guardrail フィルターを追加したり、より長期的な Memory ストラテジーを実装したりすることで、より複雑なシナリオに拡張できます。